In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('My_data_fridge_EXP.csv')

# Select relevant columns based on your dataset structure
features = data[['InTemp', 'InHumid', 'DoorState', 'Power', 'TIME', 'Temp_outside', 'Pi_consmp', 'GES(in KWh)']]

# Convert categorical variables (e.g., DoorState, Power) to numerical format
features['DoorState'] = features['DoorState'].map({'open': 1, 'CLOSED': 0})
features['Power'] = features['Power'].map({' ON': 1, ' OFF': 0})

# Normalize the numerical columns (ignoring the timestamp)
# Drop the 'TIME' column for normalization, as it's not a numerical feature
features_to_scale = features.drop(columns=['TIME'])

# Normalize the data
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_to_scale)

# Assuming that 'Energy_Source' column does not exist, we will create a dummy target (for demonstration)
# In a real scenario, this could be a decision like using RES, BES, or Grid based on some conditions
labels = np.random.choice([0, 1, 2], size=len(data))  # 0: RES, 1: BES, 2: Grid

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

print("Training and test data prepared successfully!")



Training and test data prepared successfully!


c:\Users\yamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\yamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the ANN architecture
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))  # Input layer
model.add(Dense(units=32, activation='relu'))  # Hidden layer
model.add(Dropout(0.2))  # Prevent overfitting
model.add(Dense(units=16, activation='relu'))  # Hidden layer
model.add(Dense(units=3, activation='softmax'))  # Output layer (3 classes: RES, BES, Grid)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


c:\Users\yamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=125, batch_size=36, validation_data=(X_test, y_test))


Epoch 1/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3306 - loss: 1.0987 - val_accuracy: 0.3356 - val_loss: 1.0987
Epoch 2/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3333 - loss: 1.0987 - val_accuracy: 0.3356 - val_loss: 1.0986
Epoch 3/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3305 - loss: 1.0988 - val_accuracy: 0.3281 - val_loss: 1.0987
Epoch 4/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3324 - loss: 1.0987 - val_accuracy: 0.3356 - val_loss: 1.0986
Epoch 5/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3339 - loss: 1.0987 - val_accuracy: 0.3356 - val_loss: 1.0986
Epoch 6/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3314 - loss: 1.0987 - val_accuracy: 0.3281 - val_loss: 1.0986
Epoch 7/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3330 - loss: 1.0987 - val_accuracy: 0.3356 - val_loss: 1.0986
Epoch 8/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3376 - loss: 1.0986 - val_accu

In [5]:

# Ensure 'DoorState' is properly processed and convert it to numerical format
if data['DoorState'].dtype == 'object':
    data['DoorState'] = data['DoorState'].str.strip().map({'open': 1, 'CLOSED': 0})  # Handling categorical data

# Ensure 'Power' is also converted to numerical format if needed
if data['Power'].dtype == 'object':
    data['Power'] = data['Power'].str.strip().map({'ON': 1, 'OFF': 0})  # Handling categorical data for Power

# Convert the data to numerical format and scale it
features = data[['InTemp', 'InHumid', 'DoorState', 'Power', 'Temp_outside', 'Pi_consmp', 'GES(in KWh)']]

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features)

# Function to predict energy source
def decide_energy_source(inputs):
    """Predict the energy source using the trained ANN."""
    predictions = model.predict(inputs)  # Get predictions for the entire dataset
    energy_sources = np.argmax(predictions, axis=1)  # 0 for RES, 1 for BES, 2 for Grid
    return energy_sources

# Function to adjust the temperature
def adjust_temperature(L, U, current_temp, res, fridge_power, delta=0.1):
    """Adjusts the lower limit L based on available RES."""
    if fridge_power > res:
        if current_temp > L and L < U:
            L += delta  # Increment the lower limit to save energy
            print(f"Adjusting L to: {L}")
    return L

# Predict energy sources for all 24 hours
energy_sources = decide_energy_source(scaled_data)

# Print energy source decisions for each hour
for i, source in enumerate(energy_sources):
    if source == 0:
        print(f"Hour {i}: Using solar energy (RES).")
    elif source == 1:
        print(f"Hour {i}: Using BES.")
    else:
        print(f"Hour {i}: Using Grid power.")

# Adjust the temperature based on RES and power demand for all 24 hours
L = 3  # Initial lower limit
U = 8  # Upper limit

# Loop through the 24-hour data to adjust the temperature each hour
for i in range(len(scaled_data)):
    current_temp = data.loc[i, 'InTemp']  # Current fridge temperature for hour i
    fridge_power = data.loc[i, 'Pi_consmp']  # Power demand for hour i
    res = data.loc[i, 'GES(in KWh)']  # Renewable energy available for hour i
    L = adjust_temperature(L, U, current_temp, res, fridge_power)  # Adjust L for each hour




1101/1101 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Hour 0: Using solar energy (RES).
Hour 1: Using solar energy (RES).
Hour 2: Using solar energy (RES).
Hour 3: Using solar energy (RES).
Hour 4: Using solar energy (RES).
Hour 5: Using solar energy (RES).
Hour 6: Using solar energy (RES).
Hour 7: Using solar energy (RES).
Hour 8: Using solar energy (RES).
Hour 9: Using solar energy (RES).
Hour 10: Using solar energy (RES).
Hour 11: Using solar energy (RES).
Hour 12: Using solar energy (RES).
Hour 13: Using solar energy (RES).
Hour 14: Using solar energy (RES).
Hour 15: Using solar energy (RES).
Hour 16: Using solar energy (RES).
Hour 17: Using solar energy (RES).
Hour 18: Using solar energy (RES).
Hour 19: Using solar energy (RES).
Hour 20: Using solar energy (RES).
Hour 21: Using solar energy (RES).
Hour 22: Using solar energy (RES).
Hour 23: Using solar energy (RES).
Hour 24: Using solar energy (RES).
Hour 25: Using solar energy (RES).
Hour 26: Using solar energy (RES).
Hour 27: Using solar e

In [6]:
# Reward function
def calculate_reward(L, current_temp, res, fridge_power, bes_level, U):
    reward = 0
    # Reward based on temperature within range
    if L <= current_temp <= U:
        reward += 10  # Positive reward for maintaining ideal temperature
    else:
        reward -= 10  # Negative reward if out of range

    # Reward for efficiently using RES
    if res >= fridge_power:
        reward += 20  # Positive reward for using RES
    elif bes_level > 0:
        reward += 10  # Positive reward for using BES
    else:
        reward -= 20  # Negative reward for switching to grid

    return reward

# Adjust the temperature and calculate reward for 24 hours
L = 3  # Initial lower limit
U = 7  # Upper limit

# Loop through the 24-hour data to adjust the temperature and calculate the reward
for i in range(len(scaled_data)):
    current_temp = data.loc[i, 'InTemp']  # Current fridge temperature for hour i
    fridge_power = data.loc[i, 'Pi_consmp']  # Power demand for hour i
    res = data.loc[i, 'GES(in KWh)']  # Renewable energy available for hour i
    bes_level = 50  # Example BES level for each hour (can be dynamic)
    
    # Adjust the temperature for each hour
    L = adjust_temperature(L, U, current_temp, res, fridge_power)  
    
    # Calculate the reward for each hour
    reward = calculate_reward(L, current_temp, res, fridge_power, bes_level, U)
    
    # Print the reward for each hour
    print(f"Hour {i}: Reward = {reward}")



Adjusting L to: 3.1
Hour 0: Reward = 0
Adjusting L to: 3.2
Hour 1: Reward = 0
Adjusting L to: 3.3000000000000003
Hour 2: Reward = 0
Adjusting L to: 3.4000000000000004
Hour 3: Reward = 0
Adjusting L to: 3.5000000000000004
Hour 4: Reward = 0
Adjusting L to: 3.6000000000000005
Hour 5: Reward = 0
Adjusting L to: 3.7000000000000006
Hour 6: Reward = 0
Adjusting L to: 3.8000000000000007
Hour 7: Reward = 0
Adjusting L to: 3.900000000000001
Hour 8: Reward = 0
Adjusting L to: 4.000000000000001
Hour 9: Reward = 0
Adjusting L to: 4.1000000000000005
Hour 10: Reward = 0
Adjusting L to: 4.2
Hour 11: Reward = 0
Adjusting L to: 4.3
Hour 12: Reward = 0
Adjusting L to: 4.3999999999999995
Hour 13: Reward = 0
Adjusting L to: 4.499999999999999
Hour 14: Reward = 0
Adjusting L to: 4.599999999999999
Hour 15: Reward = 0
Adjusting L to: 4.699999999999998
Hour 16: Reward = 0
Adjusting L to: 4.799999999999998
Hour 17: Reward = 0
Adjusting L to: 4.899999999999998
Hour 18: Reward = 0
Adjusting L to: 4.99999999999999

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the main dataset (My_data_fridge_EXP.csv)
data = pd.read_csv('My_data_fridge_EXP.csv')

# Load the averaged power data (averaged_power_data.csv)
power_data = pd.read_csv('averaged_power_data.csv')

# Merge the two datasets based on the TIME column
data = pd.merge(data, power_data[['TIME', 'Power_final']], on='TIME', how='left')

# Select relevant columns (including Power_final from averaged_power_data.csv)
features = data[['InTemp', 'InHumid', 'DoorState', 'Power_final', 'TIME', 'Temp_outside', 'Pi_consmp', 'GES(inKWh)']]

# Convert categorical variables (e.g., DoorState) to numerical format
features['DoorState'] = features['DoorState'].map({'open': 1, 'CLOSED': 0})

# Normalize the numerical columns (ignoring the timestamp)
# Drop the 'TIME' column for normalization, as it's not a numerical feature
features_to_scale = features.drop(columns=['TIME'])

# Normalize the data
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_to_scale)

# Assuming that 'Energy_Source' column does not exist, we will create a dummy target (for demonstration)
# In a real scenario, this could be a decision like using RES, BES, or Grid based on some conditions
labels = np.random.choice([0, 1, 2], size=len(data))  # 0: RES, 1: BES, 2: Grid

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

print("Training and test data prepared successfully!")


Training and test data prepared successfully!


C:\Users\yamin\AppData\Local\Temp\ipykernel_19996\3079627174.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['DoorState'] = features['DoorState'].map({'open': 1, 'CLOSED': 0})


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the ANN architecture
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))  # Input layer
model.add(Dense(units=32, activation='relu'))  # Hidden layer
model.add(Dropout(0.2))  # Prevent overfitting
model.add(Dense(units=16, activation='relu'))  # Hidden layer
model.add(Dense(units=3, activation='softmax'))  # Output layer (3 classes: RES, BES, Grid)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\yamin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Train the model
history = model.fit(X_train, y_train, epochs=125, batch_size=36, validation_data=(X_test, y_test))

Epoch 1/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3337 - loss: 1.0987 - val_accuracy: 0.3265 - val_loss: 1.0988
Epoch 2/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3372 - loss: 1.0986 - val_accuracy: 0.3265 - val_loss: 1.0987
Epoch 3/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3345 - loss: 1.0986 - val_accuracy: 0.3265 - val_loss: 1.0987
Epoch 4/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3265 - loss: 1.0987 - val_accuracy: 0.3265 - val_loss: 1.0989
Epoch 5/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3377 - loss: 1.0986 - val_accuracy: 0.3265 - val_loss: 1.0991
Epoch 6/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3272 - loss: 1.0989 - val_accuracy: 0.3265 - val_loss: 1.0987
Epoch 7/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3355 - loss: 1.0985 - val_accuracy: 0.3359 - val_loss: 1.0988
Epoch 8/125
783/783 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3360 - loss: 1.0986 - val_accu

In [7]:

# Ensure 'DoorState' is properly processed and convert it to numerical format
if data['DoorState'].dtype == 'object':
    data['DoorState'] = data['DoorState'].str.strip().map({'open': 1, 'CLOSED': 0})  # Handling categorical data

# Ensure 'Power' is also converted to numerical format if needed
if data['Power'].dtype == 'object':
    data['Power'] = data['Power'].str.strip().map({'ON': 1, 'OFF': 0})  # Handling categorical data for Power

# Convert the data to numerical format and scale it
features = data[['InTemp', 'InHumid', 'DoorState', 'Power', 'Temp_outside', 'Pi_consmp', 'GES(inKWh)']]

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features)

# Function to predict energy source
def decide_energy_source(inputs):
    """Predict the energy source using the trained ANN."""
    predictions = model.predict(inputs)  # Get predictions for the entire dataset
    energy_sources = np.argmax(predictions, axis=1)  # 0 for RES, 1 for BES, 2 for Grid
    return energy_sources

# Function to adjust the temperature
def adjust_temperature(L, U, current_temp, res, fridge_power, delta=0.1):
    """Adjusts the lower limit L based on available RES."""
    if fridge_power > res:
        if current_temp > L and L < U:
            L += delta  # Increment the lower limit to save energy
            print(f"Adjusting L to: {L}")
    return L

# Predict energy sources for all 24 hours
energy_sources = decide_energy_source(scaled_data)

# Print energy source decisions for each hour
for i, source in enumerate(energy_sources):
    if source == 0:
        print(f"Hour {i}: Using solar energy (RES).")
    elif source == 1:
        print(f"Hour {i}: Using BES.")
    else:
        print(f"Hour {i}: Using Grid power.")

# Adjust the temperature based on RES and power demand for all 24 hours
L = 3  # Initial lower limit
U = 8  # Upper limit

# Loop through the 24-hour data to adjust the temperature each hour
for i in range(len(scaled_data)):
    current_temp = data.loc[i, 'InTemp']  # Current fridge temperature for hour i
    fridge_power = data.loc[i, 'Pi_consmp']  # Power demand for hour i
    res = data.loc[i, 'GES(inKWh)']  # Renewable energy available for hour i
    L = adjust_temperature(L, U, current_temp, res, fridge_power)  # Adjust L for each hour

1101/1101 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Hour 0: Using BES.
Hour 1: Using BES.
Hour 2: Using BES.
Hour 3: Using BES.
Hour 4: Using BES.
Hour 5: Using BES.
Hour 6: Using BES.
Hour 7: Using BES.
Hour 8: Using BES.
Hour 9: Using BES.
Hour 10: Using BES.
Hour 11: Using BES.
Hour 12: Using BES.
Hour 13: Using BES.
Hour 14: Using BES.
Hour 15: Using BES.
Hour 16: Using BES.
Hour 17: Using BES.
Hour 18: Using BES.
Hour 19: Using BES.
Hour 20: Using BES.
Hour 21: Using BES.
Hour 22: Using BES.
Hour 23: Using BES.
Hour 24: Using BES.
Hour 25: Using BES.
Hour 26: Using BES.
Hour 27: Using BES.
Hour 28: Using BES.
Hour 29: Using BES.
Hour 30: Using BES.
Hour 31: Using BES.
Hour 32: Using BES.
Hour 33: Using BES.
Hour 34: Using BES.
Hour 35: Using BES.
Hour 36: Using BES.
Hour 37: Using BES.
Hour 38: Using BES.
Hour 39: Using BES.
Hour 40: Using BES.
Hour 41: Using BES.
Hour 42: Using BES.
Hour 43: Using BES.
Hour 44: Using BES.
Hour 45: Using BES.
Hour 46: Using BES.
Hour 47: Using BES.
Hour 48